# DSPy Question Answering Pipeline

This notebook implements a DSPy pipeline for optimizing question answering prompts.

## 1. Set up the environment

In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import dspy
from dspy.evaluate import Evaluate

In [2]:
lm = dspy.LM(
    "openai/llama-3-70b-tgi",
    temperature=0.7,
    cache=False,
    api_base=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
)
dspy.configure(lm=lm)

## 2. Load and preprocess the datasets

In [3]:
from datasets import load_dataset

train_ds = load_dataset('bdsaglam/musique-2hop', 'answerable', split='train')
val_ds = load_dataset('bdsaglam/musique-thesis', 'answerable', split='validation')
train_ds, val_ds

(Dataset({
     features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
     num_rows: 14376
 }),
 Dataset({
     features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
     num_rows: 100
 }))

In [5]:
def make_example(record):
    id2paragraph = {p['idx']: p for p in record['paragraphs']}
    supporting_paragraphs = [id2paragraph[qd['paragraph_support_idx']]['paragraph_text'] for qd in record['question_decomposition']]
    return dspy.Example(
        question=record['question'],
        paragraphs=supporting_paragraphs,
        answers=[record['answer'], *record['answer_aliases']],
    ).with_inputs('question', 'paragraphs')

In [6]:
trainset = [make_example(record) for record in train_ds][:100]
valset = [make_example(record) for record in val_ds]
dict(trainset[0])

{'question': 'When was the institute that owned The Collegian founded?',
 'paragraphs': ['The Collegian is the bi-weekly official student publication of Houston Baptist University in Houston, Texas. It was founded in 1963 as a newsletter, and adopted the newspaper format in 1990.',
  "Several private institutions of higher learning—ranging from liberal arts colleges, such as The University of St. Thomas, Houston's only Catholic university, to Rice University, the nationally recognized research university—are located within the city. Rice, with a total enrollment of slightly more than 6,000 students, has a number of distinguished graduate programs and research institutes, such as the James A. Baker Institute for Public Policy. Houston Baptist University, affiliated with the Baptist General Convention of Texas, offers bachelor's and graduate degrees. It was founded in 1960 and is located in the Sharpstown area in Southwest Houston."],
 'answers': ['1960']}

## 4. Build the Pipeline

In [7]:
class DecomposeQuestion(dspy.Signature):
    """Decompose a complex question into simpler (usually 2, 3 or 4) sub-questions . Example:
Where did the player who scored the most points in a NBA season go in the NBA Draft?
1) Who has the most points in a NBA season?
2) Where did #1 go in the NBA draft?
"""

    question: str = dspy.InputField()
    decomposition: str = dspy.OutputField(
        desc="Enumerated list of sub-questions, using '#n >>' notation for dependent questions"
    )

In [8]:
def split_subquestions(decomposition_str):
    for line in decomposition_str.split("\n"):
        if line.strip():
            parts = line.split(") ", 1)
            if len(parts) == 1:
                yield parts[0].strip()
            elif len(parts) == 2:
                yield parts[1].strip()
            else:
                raise ValueError(f"Invalid decomposition line: {line}")

In [9]:
list(split_subquestions("""
1) Who has the most points in a NBA season?
2) Where did #1 go in the NBA draft?
""".strip()))

['Who has the most points in a NBA season?',
 'Where did #1 go in the NBA draft?']

In [10]:
class GenerateAnswer(dspy.Signature):
    """Answer question based on the given context."""
    context = dspy.InputField(desc="The context to use for answering the question.")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="The factual answer to the question.")

In [11]:
class SimplifiedBaleen(dspy.Module):
    def __init__(self):
        super().__init__()
        self.qdecomp = dspy.Predict(DecomposeQuestion)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question, paragraphs):
        # Decompose the question into sub-questions
        sub_questions = list(split_subquestions(self.qdecomp(question=question).decomposition))
        dspy.Suggest(len(sub_questions) == len(paragraphs), f"This question is composed of {len(paragraphs)} sub-questions, but you generated {len(sub_questions)} sub-questions.")

        # Answer each sub-question in sequence  
        sub_answers = []
        for i, (paragraph, sub_q) in enumerate(zip(paragraphs, sub_questions)):
            if i == 0:
                pred = self.generate_answer(context=paragraph, question=sub_q)
                sub_answers.append(pred.answer)
            else:
                sub_q = sub_q.replace(f"#{i}", sub_answers[i-1])
                pred = self.generate_answer(context=paragraph, question=sub_q)
                sub_answers.append(pred.answer)

        return dspy.Prediction(answer=sub_answers[-1], sub_questions=sub_questions, sub_answers=sub_answers)

In [12]:
# Initialize the uncompiled QA module
uncompiled_qa = SimplifiedBaleen()

## 5. Define the optimization metric

In [13]:
from dspy.evaluate import answer_exact_match_str

def evaluate_answer(example, pred, trace=None):
    return answer_exact_match_str(pred.answer, example.answers, frac=0.8)

In [14]:
evaluate_qa = Evaluate(devset=valset, metric=evaluate_answer, num_threads=8, display_progress=True, return_outputs=True)

In [15]:
# Evaluate the uncompiled question decomposition module
uncompiled_score, uncompiled_results = evaluate_qa(uncompiled_qa)
print(f"Uncompiled Question Answering Score: {uncompiled_score}")

  0%|          | 0/100 [00:00<?, ?it/s]

Average Metric: 60 / 100  (60.0): 100%|██████████| 100/100 [03:56<00:00,  2.36s/it]

Uncompiled Question Answering Score: 60.0


## 6. Implement the optimization process

In [16]:
from dspy.teleprompt import BootstrapFewShot, LabeledFewShot, BootstrapFewShotWithRandomSearch

# Set up the teleprompter
teleprompter = BootstrapFewShotWithRandomSearch(
    metric=evaluate_answer, 
    max_bootstrapped_demos=8, 
)

# Compile and optimize the QA module
compiled_qa = teleprompter.compile(uncompiled_qa, trainset=trainset)
compiled_qa.save('compiled-qa.json')

print("QA module compiled and optimized")

Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 16 candidate sets.


  0%|          | 0/100 [00:00<?, ?it/s]

Average Metric: 48.0 / 100  (48.0): 100%|██████████| 100/100 [04:02<00:00,  2.42s/it]


New best score: 48.0 for seed -3
Scores so far: [48.0]
Best score so far: 48.0


Average Metric: 51.0 / 100  (51.0): 100%|██████████| 100/100 [03:34<00:00,  2.15s/it]


...

Scores so far: [48.0, 51.0, 54.0, 54.0, 55.0, 55.0, 55.0, 54.0, 59.0, 50.0, 56.0, 52.0]
Best score so far: 59.0


 11%|█         | 11/100 [01:40<13:29,  9.10s/it]


Bootstrapped 8 full traces after 12 examples in round 0.


Average Metric: 45.0 / 84  (53.6):  84%|████████▍ | 84/100 [05:50<00:51,  3.21s/it]2024-10-09T21:53:50.637741Z [warning  ] Received SIGINT. Cancelling evaluation. [dspy.evaluate.evaluate] filename=evaluate.py lineno=105


KeyboardInterrupt: 

In [ ]:
compiled_qa.save('compiled-qa.json')

## 7. Evaluate the pipeline

In [25]:
# Evaluate the compiled QA module
compiled_score, compiled_results = evaluate_qa(compiled_qa)
print(f"Compiled QA Module Score: {compiled_score}")

Average Metric: 1 / 10  (10.0): 100%|██████████| 10/10 [00:12<00:00,  1.21s/it]
Uncompiled QA Module Score: 10.0
Average Metric: 3 / 10  (30.0): 100%|██████████| 10/10 [00:12<00:00,  1.25s/it]
Compiled QA Module Score: 30.0


## 8. (Optional) Error Analysis

In [ ]:
def present_errors(results):
    errors = [(example, pred) for example, pred, score in results if score < 1.0] 
    for example, pred in errors:
        print(f"Question: {example.question}")
        print(f"Context: {example.context}")
        print(f"Groundtruth Answers: {example.answers}")
        print(f"Predicted Answer: {pred.answer}")
        print()

In [ ]:
print("Error analysis for uncompiled program:")
present_errors(uncompiled_results)

print("Error analysis for compiled program:")
present_errors(compiled_results)